In [1]:
import pandas as pd
import re
import string
import seaborn as sns
from pyarabic.araby import strip_harakat
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import nltk 
from string import punctuation
import heapq  

In [2]:
df_train = pd.read_csv("datasets/101121_news_source_HSEP_train.csv" )
df_test = pd.read_csv("datasets/101121_news_source_HSEP_test.csv" )
df = pd.concat([df_train, df_test], axis=0)
df.drop(columns=["ID"], inplace=True)
df = df.reset_index(drop=True)


In [3]:
df.shape

(12225, 2)

In [4]:
df.head(10)

,Subject,Content
0,سواريز يحسم الجدل الذي تسبب به احتفاله بالهدف,<p>متابعة – مظفر إسماعيل</p><p> </p><p>نفى “ل...
1,ماذا يفعل القرنفل على الريق؟,<p>خبرني - القرنفل على الريق هي ذات الفوائد ال...
2,لن تتوقعها.. حيلة ذكية تجعل ثمرة الأفوكادو تنض...,<p>متابعة – علي معلا:</p><p>يحب الجميع تناول ف...
3,السعودية تستضيف اليابان ضمن 6 مباريات فى تصفيا...,تقام اليوم الخميس، 6 مواجهات ضمن مباريات الجول...
4,انتهاء التصويت.. صناديق الاقتراع العام تغلق في...,أعلنت المفوضية العليا للانتخابات في العراق عن ...
5,إنريكى: مواجهة إسبانيا ضد إيطاليا صعبة وسنلعب ...,أكد لويس انريكي المدير الفني لمنتخب اسبانيا عل...
6,"تنسيقية شباب الأحزاب تعد تقريرا بعنوان"" أكتوبر...",أعدت تنسيقية شباب الأحزاب والسياسيين تقريرا تح...
7,خلال اجتماعها ..”المواد الغذائية” ترفع تقريرًا...,<p>رفعت شعبة المواد الغذائية بغرفة القاهرة تقر...
8,السعودية تعطل الكمبيوتر الياباني في تصفيات مون...,<p>خبرني - حقق المنتخب السعودي فوزا ثمينا على...
9,اكرم يسرى جناح نادى الزمالك يحصل على لقب هداف ...,<p>احتل لاعب نادى الزمالك اكرم يسرى صدارة هداف...


# Cleaning Data

In [5]:
arabic_punctuations = '''«»`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations
punctuations_list

'«»`÷×؛<>_()*&^%][ـ،/:"؟.,\'{}~¦+|!”…“–ـ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [6]:
def clean(text):
    output = re.sub(r'\s*[A-Za-z]+\b', ' ' , text) #Remove english letters
    output = strip_harakat(output) #Remove harakat   
    translator = str.maketrans(' ',' ', punctuations_list) #remove arabic and english punctuations
    output = output.translate(translator)
    output = " ".join(output.split()) #remove extra spaces
    output = re.sub('\w*\d\w*', ' ', output)# Remove numbers
    
###delete_links
    # pettern  = r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
    # output = re.sub(pettern, ' ', text)
    
###delete_repeated_characters
    # pattern  = r'(.)\1{2,}'
    # output = re.sub(pattern, r"\1\1", text)
    
###replace_letters
    # replace = {"أ": "ا","ة": "ه","إ": "ا","آ": "ا","": ""}
    # replace = dict((re.escape(k), v) for k, v in replace.items()) 
    # pattern = re.compile("|".join(replace.keys()))
    # output = pattern.sub(lambda m: replace[re.escape(m.group(0))], text)
    
###remove_vowelization
    # vowelization = re.compile(""" ّ|َ|ً|ُ|ٌ|ِ|ٍ|ْ|ـ""", re.VERBOSE)
    # output = re.sub(vowelization, '', text)
    return output

In [7]:
def apply_clean_dataframe(dataframe):
    dataframe.Content = dataframe.Content.map(clean)
    dataframe.Subject = dataframe.Subject.map(clean)

    dataframe["Text"] = dataframe.Subject + " " + dataframe.Content
    return dataframe

In [8]:
def apply_clean_string(input_text):
    out_text = clean(input_text)
    return out_text

In [9]:
apply_clean_dataframe(df)
df.tail()


,Subject,Content,Text
6108,مسيرة النفير الكبير اليوم رفضا لالانقلاب في تونس,في سياق حملة تحركات متواصلة حملت شعار يوم الحس...,مسيرة النفير الكبير اليوم رفضا لالانقلاب في تو...
6109,إنقاذ حياة طفلة كانت تعاني من ورم نادر,نجح فريق جراحة الأطفال والتدخل السريع بصحة جاز...,إنقاذ حياة طفلة كانت تعاني من ورم نادر نجح فري...
6110,إصابة المتحدث باسم الخارجية الأمريكية نيد براي...,أفادت فضائية العربية في خبر عاجل لها منذ قليل ...,إصابة المتحدث باسم الخارجية الأمريكية نيد براي...
6111,سعر الدولار اليوم الإثنين سبتمبر في العراق,هبطت أسعار صرف الدولار مقابل الدينار العراقي ف...,سعر الدولار اليوم الإثنين سبتمبر في العراق...
6112,لا تصدقها أبدا أشهر خرافات عن النوم ليس لا ت...,متابعة علي معلا النوم له دور حيوي ورئيسي في حي...,لا تصدقها أبدا أشهر خرافات عن النوم ليس لا ت...


# Saving cleaned data

In [10]:
df.to_csv("datasets/clean_data.csv")

## Arabic Stopwords

In [11]:
# nltk.download("stopwords")
arb_stopwords = set(nltk.corpus.stopwords.words("arabic"))
arb_stopwords

{'ء',
 'ءَ',
 'آ',
 'آب',
 'آذار',
 'آض',
 'آمينَ',
 'آناء',
 'آنفا',
 'آه',
 'آها',
 'آهاً',
 'آهٍ',
 'آهِ',
 'آي',
 'أ',
 'أبدا',
 'أبريل',
 'أبو',
 'أبٌ',
 'أجل',
 'أجمع',
 'أحد',
 'أخبر',
 'أخذ',
 'أخو',
 'أخٌ',
 'أربع',
 'أربعاء',
 'أربعة',
 'أربعمئة',
 'أربعمائة',
 'أرى',
 'أسكن',
 'أصبح',
 'أصلا',
 'أضحى',
 'أطعم',
 'أعطى',
 'أعلم',
 'أغسطس',
 'أف',
 'أفريل',
 'أفعل به',
 'أفٍّ',
 'أقبل',
 'أقل',
 'أكتوبر',
 'أكثر',
 'أل',
 'ألا',
 'ألف',
 'ألفى',
 'أم',
 'أما',
 'أمام',
 'أمامك',
 'أمامكَ',
 'أمد',
 'أمس',
 'أمسى',
 'أمّا',
 'أن',
 'أنا',
 'أنبأ',
 'أنت',
 'أنتم',
 'أنتما',
 'أنتن',
 'أنتِ',
 'أنشأ',
 'أنى',
 'أنًّ',
 'أنّى',
 'أهلا',
 'أو',
 'أوت',
 'أوشك',
 'أول',
 'أولئك',
 'أولاء',
 'أولالك',
 'أوه',
 'أوّهْ',
 'أى',
 'أي',
 'أيا',
 'أيار',
 'أيضا',
 'أيلول',
 'أين',
 'أينما',
 'أيها',
 'أيّ',
 'أيّان',
 'أُفٍّ',
 'ؤ',
 'إحدى',
 'إذ',
 'إذا',
 'إذاً',
 'إذما',
 'إذن',
 'إزاء',
 'إلا',
 'إلى',
 'إليك',
 'إليكم',
 'إليكما',
 'إليكن',
 'إليكنّ',
 'إليكَ',
 'إلَيْكَ',
 'إلّا',


## Applying TF-IDF vectorizer

In [12]:
documents = df.Text.copy()

vectorizer = TfidfVectorizer(stop_words=arb_stopwords,min_df=0.01) #,ngram_range=(1,3),min_df=0.01
X_fit = vectorizer.fit_transform(documents)


c:\Users\MedOo\project\lib\site-packages\sklearn\feature_extraction\text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آمين', 'أب', 'أخ', 'أفعل', 'أفعله', 'ؤلاء', 'إل', 'إم', 'ات', 'اتان', 'ارتد', 'ان', 'انفك', 'برح', 'تان', 'تبد', 'تحو', 'تعل', 'حد', 'حم', 'حي', 'خب', 'ذار', 'سيما', 'صه', 'ظل', 'ظن', 'عد', 'قط', 'مر', 'مكان', 'مكانكن', 'نب', 'هات', 'هب', 'واها', 'وراء'] not in stop_words.
  % sorted(inconsistent)


## Drawing Elbow

In [75]:
# #Drawing Elbow
# import matplotlib.pyplot as plt
# wcss = []
# for i in range(3,14):
#     kmeans = KMeans(n_clusters=i, init='random',max_iter=500,n_init=20,random_state=1)
#     kmeans.fit(X_fit)
#     wcss.append(kmeans.inertia_)
# plt.plot(range(3,14),wcss)
# plt.title('The Elbow Method')
# plt.xlabel('Number of clusters')
# plt.ylabel('WCSS')
# plt.savefig('elbow.png')
# plt.show()

In [ ]:
#Converting to df
columns = vectorizer.get_feature_names()

        #using to dense to convert sparse to normal matrix
tf_idf = pd.DataFrame(X_fit.todense(), columns=columns, index=documents)
tf_idf.head(3)

## Applying K-means

In [76]:
true_k = 6
# model = KMeans(n_clusters=true_k, init='k-means++',random_state=60, max_iter=100, n_init=1)
model = KMeans(n_clusters=true_k, init='random',max_iter=700,n_init=20,random_state=1)
model.fit(X_fit)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),

Top terms per cluster:
Cluster 0:
 الدوري
 فى
 الفريق
 منتخب
 المباراة
 مباراة
 محمد
 المنتخب
 الزمالك
 صلاح
Cluster 1:
 سعر
 للشراء
 للبيع
 المركزي
 الدولار
 الدينار
 بنك
 صرف
 متوسط
 ليبيا
Cluster 2:
 يمكن
 الجسم
 الدم
 تناول
 القلب
 الإصابة
 يساعد
 النوم
 بشكل
 الوزن
Cluster 3:
 الذهب
 سعر
 عيار
 أسعار
 جرام
 دينارا
 للأوقية
 تعاملات
 الدولار
 اليوم
Cluster 4:
 حالة
 كورونا
 إصابة
 وفاة
 جديدة
 الصحة
 تسجيل
 إجمالي
 بفيروس
 عدد
Cluster 5:
 خلال
 رئيس
 المملكة
 المتحدة
 كورونا
 العام
 مجلس
 الرئيس
 بن
 الخارجية


c:\Users\MedOo\project\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### Results
> We can notice that topics are:
>> Cluster 0: Sport<br>
>> Cluster 1: Economy<br>
>> Cluster 2: Health<br>
>> Cluster 3: Economy<br>
>> Cluster 4: Health<br>
>> Cluster 5: politics

In [77]:
prediction = model.predict(X_fit)
print(prediction)

[0 2 2 ... 5 1 2]


In [80]:
filename = "models/ar_kmeans.pkl"
pickle.dump(model, open(filename,"wb"))    #saving the model

In [13]:
def add_cluster_col(dataframe,xx):
    dataframe["Topic_AR"] = xx
    dataframe.Topic_AR.replace({0:"رياضة", 1:"إقتصاد", 2:"صحة", 3:"إقتصاد", 4:"صحة" ,5:"سياسة"}, inplace=True)

    dataframe["Topic_EN"] = xx
    dataframe.Topic_EN.replace({0: "sport", 1:"Eco", 2:"Health", 3:"Eco", 4:"Health", 5:"politics" }, inplace=True)

In [1]:
add_cluster_col(df,prediction)
df.head(30)

NameError: name 'add_cluster_col' is not defined

In [15]:
df.to_csv("datasets/clustered.csv")

In [16]:
sns.countplot(data=df,x='Topic_EN')

ValueError: Could not interpret input 'Topic_EN'

In [17]:
def tfidf_features(X_train, X_test):
    tfidf_vectorizer = TfidfVectorizer(stop_words=arb_stopwords,min_df=0.01)
    tfidf_vectorizer_m = tfidf_vectorizer.fit(X_train)
    X_train = tfidf_vectorizer_m.transform(X_train)
    X_test = tfidf_vectorizer_m.transform(X_test)
    return X_train, X_test


# Text Summarization

In [18]:
def nltk_summarizer_text(text, number_of_sentence):
    stopWords = set(nltk.corpus.stopwords.words("arabic") + nltk.corpus.stopwords.words("english"))
    word_frequencies = {}  
    for word in nltk.word_tokenize(text):  
        if word not in stopWords:
            if word not in punctuation:
                if word not in word_frequencies.keys():
                    word_frequencies[word] = 1
                else:
                    word_frequencies[word] += 1

    maximum_frequncy = max(word_frequencies.values())

    for word in word_frequencies.keys():  
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

    sentence_list = nltk.sent_tokenize(text)
    sentence_scores = {}  
    for sent in sentence_list:  
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]

    summary_sentences = heapq.nlargest(number_of_sentence, sentence_scores, key=sentence_scores.get)

    summary = ' '.join(summary_sentences)  
    return summary

In [19]:
# categories = ['Sports', 'Economy', 'Health', 'Economy', 'Health', 'Politics']

# def summerize_category(input_text, statements, model_name):
#     summary_text = nltk_summarizer(input_text, statements)
#     print("----------------------------------------------------------------------------")
#     print("Text summary")
#     print("----------------------------------------------------------------------------")
#     print(summary_text)
#     print("----------------------------------------------------------------------------")
#     input_text_arr = [apply_clean_string(input_text)]
  
#     X_trans = vectorizer.transform(input_text_arr)

#     text_predection = model_name.predict(X_trans)
#     text_category = categories[text_predection[0]]
#     print("Text category:", text_category)
#     print("----------------------------------------------------------------------------")

In [20]:
categories = ['Sports', 'Economy', 'Health', 'Economy', 'Health', 'Politics']

def summerize_category_text(input_text, statements, model_name):
    summary_text = nltk_summarizer_text(input_text, statements)
    print("----------------------------------------------------------------------------")
    print("Text summary")
    print("----------------------------------------------------------------------------")
    print(summary_text)
    print("----------------------------------------------------------------------------")
    input_text = str(input_text)
    input_text_arr = [apply_clean_string(input_text)]
  
    X_trans ,y_trans = tfidf_features(documents,input_text_arr)

    text_predection = model_name.predict(y_trans)
    text_category = categories[text_predection[0]]
    print("Text category:", text_category)
    print("----------------------------------------------------------------------------")
    return summary_text,text_category

In [21]:
ar_test_1 = "التكنولوجيا هي كلمة ليست عربية ولكن لها مرادف معرب,\
أقترحه مجمع اللغة العربية بدمشق, واعتمدته الجامعة العربية وبعض الدول العربية, ولكن ليس كلها. و هي كلمة شائعة لها أكثر من مصطلح فالتكنولوجيا \
هي علم تطبيقي يهتم بدراسة الإضافات والتطور في العديد من المجالات مثل الصناعات و الفنون والحرف وكل ما يتعلق بها من مواد ووسائل مستعمل.\
والمفهوم الشّائع لمصطلح التّكنولوجيا هو استعمال الكمبيوتر والأجهزة الحديثة، \
وهذه النّظرة محدودة الرؤية ، فالكمبيوتر نتيجة من نتائج التكنولوجيا،  بينما التكنولوجيا الّتي يقصدها هذا المقرّر هي طريقة للتّفكير، وحلّ المشكلات، وهي أسلوب التّفكير الّذي\
يوصل الفرد إلى النتائج المرجوّة أي إنّها وسيلة وليست نتيجة، وإنّها طريقة التّفكير في استخدام المعارف، والمعلومات، والمهارات، بهدف الوصول إلى نتائج لإشباع حاجة الإنسان وزيادة قدراته.\
 أوّل فائدة تستحقّ الذكر هي ما أسهمت بهِ الثورة العلميّة والتكنولوجيّة في مجال مٌكافحة الأمراض، وتطوير المضادّات الحيوية وهو ما يُدعى بالتكنولوجيا الطبيّة الحيويّة، وأيضاُ مساهمة الأجهزة\
 الطبيّة الحديثة في مجالات تخطيط القلب والتصوير الإشعاعي في تشخيص الأمراض، والوصول إلى نتائج مُذهلة في هذا النطاق، أضِف إلى ذلك إدخال التكنولوجيا في إجراء العمليّات الجراحيةّ\
 المعقّدة والدقيقة والجراحة بالمنظار والقسطرة، وكذلك تكنولوجيا صناعة الدواء مما ساعدَ كثيراً في شفاء الحالات المُستعصيّة، وتسهيل العمل على الطاقم البشري من أطبّاء وممرضين وصيادلة.\
 كما نذكُر فائدة التكنولوجيا في التسهيل على الناس في التنقّل من خلال وسائل النقل والمواصلات الحديثة؛ كظهُور القطارات الكهربائية، والطائرات، وأنظمة النقل البحريّ المتطوّرة.\
 وأيضا تساعد التكنولوجيا علي تسهيل مهامّ البحث العلميّ والوصول إلى المعلومة بأقصر وقت وأقلّ تكلفة من السابق؛ حيث توفّر شبكة الإنترنت."

In [22]:
model_test = pickle.load(open('models/ar_kmeans.pkl', 'rb'))

In [24]:
summerize_category_text(ar_test_1,1,model_test)


----------------------------------------------------------------------------
Text summary
----------------------------------------------------------------------------
التكنولوجيا هي كلمة ليست عربية ولكن لها مرادف معرب,أقترحه مجمع اللغة العربية بدمشق, واعتمدته الجامعة العربية وبعض الدول العربية, ولكن ليس كلها.
----------------------------------------------------------------------------
Text category: Politics
----------------------------------------------------------------------------


('التكنولوجيا هي كلمة ليست عربية ولكن لها مرادف معرب,أقترحه مجمع اللغة العربية بدمشق, واعتمدته الجامعة العربية وبعض الدول العربية, ولكن ليس كلها.',
 'Politics')

## Prediction

In [20]:
test_1= pd.read_csv('datasets/test.csv')
test_1
# test_1 = pd.read_csv('scraped_news.csv')
# test_1

,Subject,Content
0,"'غلق كلى لشارع الهرم بتقاطعه مع ""العمدة الجديد...",'علنت محافظة الجيزة، استكمال تسليم مواقع محطات...
1,'\r\nرئيس الوزراء يوافق على منح الجنسية المصري...,'وافق الدكتور مصطفى مدبولى ، رئيس الوزراء على ...
2,'قرار حكومى باعتبار مشروع إنشاء محور الفريق كم...,'أصدر الدكتور مصطفى مدبولى، رئيس الوزراء قرار ...
3,'\r\nأجواء معتدلة وشبورة كثيفة ونشاط رياح.. تف...,'كشفت الهيئة العامة للأرصاد الجوية، تفاصيل حال...
4,'\r\nدراسة دولية: كورونا تتسبب فى فقدان 28 ملي...,'قالت صحيفة واشنطن بوست إن دراسة دولية، تم الك...
5,'ماذا يفعل القرنفل على الريق','خبرني القرنفل على الريق هي ذات الفوائد التي ي...
6,'محمد محمود الخيار الرابع لموسيمانى فى وسط ملع...,'رغم اقتناع الجنوب أفريقي بيتسو موسيماني، المد...
7,'\r\nأسامة الصعيدى يكشف تفاصيل عقد محمد صلاح إ...,'كشف المستشار أسامة الصعيدى عضو مجلس إدارة الم...
8,'\r\nمجلس الشيوخ يقر المادة المنظمة للتعريفات ...,'وافق مجلس الشيوخ، برئاسة المستشار عبد الوهاب ...
9,"كواليس غرفة الملابس.. عن أزمة ""كوع"" صالح جمعة ...",دائماً ما تشهد غرفة خلع ملابس الأندية العديد م...


In [55]:
# test_1 = test_1[['Subject','Content']]

In [56]:
apply_clean_dataframe(test_1)

,Subject,Content,Text
0,غلق كلى لشارع الهرم بتقاطعه مع العمدة الجديد ل...,علنت محافظة الجيزة استكمال تسليم مواقع محطات م...,غلق كلى لشارع الهرم بتقاطعه مع العمدة الجديد ل...
1,رئيس الوزراء يوافق على منح الجنسية المصرية لعد...,وافق الدكتور مصطفى مدبولى رئيس الوزراء على منح...,رئيس الوزراء يوافق على منح الجنسية المصرية لعد...
2,قرار حكومى باعتبار مشروع إنشاء محور الفريق كما...,أصدر الدكتور مصطفى مدبولى رئيس الوزراء قرار با...,قرار حكومى باعتبار مشروع إنشاء محور الفريق كما...
3,أجواء معتدلة وشبورة كثيفة ونشاط رياح تفاصيل حا...,كشفت الهيئة العامة للأرصاد الجوية تفاصيل حالة ...,أجواء معتدلة وشبورة كثيفة ونشاط رياح تفاصيل حا...
4,دراسة دولية كورونا تتسبب فى فقدان مليون سنة ...,قالت صحيفة واشنطن بوست إن دراسة دولية تم الكشف...,دراسة دولية كورونا تتسبب فى فقدان مليون سنة ...
5,ماذا يفعل القرنفل على الريق,خبرني القرنفل على الريق هي ذات الفوائد التي يج,ماذا يفعل القرنفل على الريق خبرني القرنفل على ...
6,محمد محمود الخيار الرابع لموسيمانى فى وسط ملعب...,رغم اقتناع الجنوب أفريقي بيتسو موسيماني المدير...,محمد محمود الخيار الرابع لموسيمانى فى وسط ملعب...
7,أسامة الصعيدى يكشف تفاصيل عقد محمد صلاح إلى با...,كشف المستشار أسامة الصعيدى عضو مجلس إدارة المق...,أسامة الصعيدى يكشف تفاصيل عقد محمد صلاح إلى با...
8,مجلس الشيوخ يقر المادة المنظمة للتعريفات بقانو...,وافق مجلس الشيوخ برئاسة المستشار عبد الوهاب عب...,مجلس الشيوخ يقر المادة المنظمة للتعريفات بقانو...
9,كواليس غرفة الملابس عن أزمة كوع صالح جمعة لمعل...,دائما ما تشهد غرفة خلع ملابس الأندية العديد من...,كواليس غرفة الملابس عن أزمة كوع صالح جمعة لمعل...


In [58]:
testt = test_1.Text.copy()

X_trans = vectorizer.transform(testt)

pediction_result = model_test.predict(X_trans)
pediction_result

array([5, 5, 5, 5, 5, 5, 0, 0, 5, 0, 0, 5, 5, 1])

In [59]:
add_cluster_col(test_1,pediction_result)
test_1.to_csv("datasets/tested_labled.csv")
test_1.tail(45)

,Subject,Content,Text,Topic_AR,Topic_EN
0,غلق كلى لشارع الهرم بتقاطعه مع العمدة الجديد ل...,علنت محافظة الجيزة استكمال تسليم مواقع محطات م...,غلق كلى لشارع الهرم بتقاطعه مع العمدة الجديد ل...,سياسة,politics
1,رئيس الوزراء يوافق على منح الجنسية المصرية لعد...,وافق الدكتور مصطفى مدبولى رئيس الوزراء على منح...,رئيس الوزراء يوافق على منح الجنسية المصرية لعد...,سياسة,politics
2,قرار حكومى باعتبار مشروع إنشاء محور الفريق كما...,أصدر الدكتور مصطفى مدبولى رئيس الوزراء قرار با...,قرار حكومى باعتبار مشروع إنشاء محور الفريق كما...,سياسة,politics
3,أجواء معتدلة وشبورة كثيفة ونشاط رياح تفاصيل حا...,كشفت الهيئة العامة للأرصاد الجوية تفاصيل حالة ...,أجواء معتدلة وشبورة كثيفة ونشاط رياح تفاصيل حا...,سياسة,politics
4,دراسة دولية كورونا تتسبب فى فقدان مليون سنة ...,قالت صحيفة واشنطن بوست إن دراسة دولية تم الكشف...,دراسة دولية كورونا تتسبب فى فقدان مليون سنة ...,سياسة,politics
5,ماذا يفعل القرنفل على الريق,خبرني القرنفل على الريق هي ذات الفوائد التي يج,ماذا يفعل القرنفل على الريق خبرني القرنفل على ...,سياسة,politics
6,محمد محمود الخيار الرابع لموسيمانى فى وسط ملعب...,رغم اقتناع الجنوب أفريقي بيتسو موسيماني المدير...,محمد محمود الخيار الرابع لموسيمانى فى وسط ملعب...,رياضة,sport
7,أسامة الصعيدى يكشف تفاصيل عقد محمد صلاح إلى با...,كشف المستشار أسامة الصعيدى عضو مجلس إدارة المق...,أسامة الصعيدى يكشف تفاصيل عقد محمد صلاح إلى با...,رياضة,sport
8,مجلس الشيوخ يقر المادة المنظمة للتعريفات بقانو...,وافق مجلس الشيوخ برئاسة المستشار عبد الوهاب عب...,مجلس الشيوخ يقر المادة المنظمة للتعريفات بقانو...,سياسة,politics
9,كواليس غرفة الملابس عن أزمة كوع صالح جمعة لمعل...,دائما ما تشهد غرفة خلع ملابس الأندية العديد من...,كواليس غرفة الملابس عن أزمة كوع صالح جمعة لمعل...,رياضة,sport


In [27]:
test_1= pd.read_csv('datasets/test.csv')
test_1
# test_1 = pd.read_csv('scraped_news.csv')
# test_1

,Subject,Content
0,"'غلق كلى لشارع الهرم بتقاطعه مع ""العمدة الجديد...",'علنت محافظة الجيزة، استكمال تسليم مواقع محطات...
1,'\r\nرئيس الوزراء يوافق على منح الجنسية المصري...,'وافق الدكتور مصطفى مدبولى ، رئيس الوزراء على ...
2,'قرار حكومى باعتبار مشروع إنشاء محور الفريق كم...,'أصدر الدكتور مصطفى مدبولى، رئيس الوزراء قرار ...
3,'\r\nأجواء معتدلة وشبورة كثيفة ونشاط رياح.. تف...,'كشفت الهيئة العامة للأرصاد الجوية، تفاصيل حال...
4,'\r\nدراسة دولية: كورونا تتسبب فى فقدان 28 ملي...,'قالت صحيفة واشنطن بوست إن دراسة دولية، تم الك...
5,'ماذا يفعل القرنفل على الريق','خبرني القرنفل على الريق هي ذات الفوائد التي ي...
6,'محمد محمود الخيار الرابع لموسيمانى فى وسط ملع...,'رغم اقتناع الجنوب أفريقي بيتسو موسيماني، المد...
7,'\r\nأسامة الصعيدى يكشف تفاصيل عقد محمد صلاح إ...,'كشف المستشار أسامة الصعيدى عضو مجلس إدارة الم...
8,'\r\nمجلس الشيوخ يقر المادة المنظمة للتعريفات ...,'وافق مجلس الشيوخ، برئاسة المستشار عبد الوهاب ...
9,"كواليس غرفة الملابس.. عن أزمة ""كوع"" صالح جمعة ...",دائماً ما تشهد غرفة خلع ملابس الأندية العديد م...


In [28]:
test_2=apply_clean_dataframe(test_1)

In [24]:
testt = test_1.copy()

In [107]:
print(type(testt.Text))
# testt[:,:0]
x = str(testt.Text)
print(type(x))
print(x)

<class 'pandas.core.series.Series'>
<class 'str'>
0     غلق كلى لشارع الهرم بتقاطعه مع العمدة الجديد ل...
1     رئيس الوزراء يوافق على منح الجنسية المصرية لعد...
2     قرار حكومى باعتبار مشروع إنشاء محور الفريق كما...
3     أجواء معتدلة وشبورة كثيفة ونشاط رياح تفاصيل حا...
4     دراسة دولية كورونا تتسبب فى فقدان مليون سنة من...
5     ماذا يفعل القرنفل على الريق خبرني القرنفل على ...
6     محمد محمود الخيار الرابع لموسيمانى فى وسط ملعب...
7     أسامة الصعيدى يكشف تفاصيل عقد محمد صلاح إلى با...
8     مجلس الشيوخ يقر المادة المنظمة للتعريفات بقانو...
9     كواليس غرفة الملابس عن أزمة كوع صالح جمعة لمعل...
10    التشكيل المتوقع للزمالك أمام البنك الأهلى زيزو...
11    التصديرى للصناعات الهندسية يستقبل أكبر بعثة مش...
12    زيادة فى معدلات الصادرات المصرية خلال أشهر الأ...
13    سعر الدينار الكويتى اليوم الاثنين انخفض سعر ال...
Name: Text, dtype: object


In [25]:
def nltk_summarizer_dataframe(dataframe, number_of_sentence):
    # stopWords = set(nltk.corpus.stopwords.words("arabic") + nltk.corpus.stopwords.words("english"))
    # word_frequencies = {}
    dataframe['summary'] = dataframe.Text.map(lambda s: nltk_summarizer_text(s,number_of_sentence))  
    return dataframe


In [29]:
df_testtt = nltk_summarizer_dataframe(test_2,2)

In [30]:
df_testtt.head()

,Subject,Content,Text,summary
0,غلق كلى لشارع الهرم بتقاطعه مع العمدة الجديد ل...,علنت محافظة الجيزة استكمال تسليم مواقع محطات م...,غلق كلى لشارع الهرم بتقاطعه مع العمدة الجديد ل...,
1,رئيس الوزراء يوافق على منح الجنسية المصرية لعد...,وافق الدكتور مصطفى مدبولى رئيس الوزراء على منح...,رئيس الوزراء يوافق على منح الجنسية المصرية لعد...,رئيس الوزراء يوافق على منح الجنسية المصرية لعد...
2,قرار حكومى باعتبار مشروع إنشاء محور الفريق كما...,أصدر الدكتور مصطفى مدبولى رئيس الوزراء قرار با...,قرار حكومى باعتبار مشروع إنشاء محور الفريق كما...,
3,أجواء معتدلة وشبورة كثيفة ونشاط رياح تفاصيل حا...,كشفت الهيئة العامة للأرصاد الجوية تفاصيل حالة ...,أجواء معتدلة وشبورة كثيفة ونشاط رياح تفاصيل حا...,
4,دراسة دولية كورونا تتسبب فى فقدان مليون سنة ...,قالت صحيفة واشنطن بوست إن دراسة دولية تم الكشف...,دراسة دولية كورونا تتسبب فى فقدان مليون سنة ...,


In [52]:
def summerize_category_dataframe(dataframe, statements, model_name):
    input_dataframe =apply_clean_dataframe(dataframe)
    X_trans ,y_trans = tfidf_features(documents,input_dataframe)
    # bb=vectorizer.transform(testt) #tf-idf vector
    # print(X_trans.shape)
    text_predection = model_name.predict(y_trans)
    # print(text_predection)
    add_cluster_col(input_dataframe,text_predection)
    summary = nltk_summarizer_dataframe(input_dataframe, statements)

    return summary,input_dataframe.Topic_AR

In [53]:

summerize_category_dataframe(testt, 2, model_test)

ValueError: Length of values (3) does not match length of index (14)

In [97]:
x = nltk_summarizer_dataframe(test_1,3)

# Summarization using bert model

In [14]:
# Import generic wrappers
from transformers import AutoModel, AutoTokenizer 


# Define the model repo
model_name = "asafaya/bert-base-arabic" 


# Download pytorch model
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


# Transform input tokens 
inputs = tokenizer("التكنولوجيا هي كلمة ليست عربية ولكن لها مرادف معرب,\
أقترحه مجمع اللغة العربية بدمشق, واعتمدته الجامعة العربية وبعض الدول العربية, ولكن ليس كلها. و هي كلمة شائعة لها أكثر من مصطلح فالتكنولوجيا \
هي علم تطبيقي يهتم بدراسة الإضافات والتطور في العديد من المجالات مثل الصناعات و الفنون والحرف وكل ما يتعلق بها من مواد ووسائل مستعمل.\
والمفهوم الشّائع لمصطلح التّكنولوجيا هو استعمال الكمبيوتر والأجهزة الحديثة، \
وهذه النّظرة محدودة الرؤية ، فالكمبيوتر نتيجة من نتائج التكنولوجيا،  بينما التكنولوجيا الّتي يقصدها هذا المقرّر هي طريقة للتّفكير، وحلّ المشكلات، وهي أسلوب التّفكير الّذي\
يوصل الفرد إلى النتائج المرجوّة أي إنّها وسيلة وليست نتيجة، وإنّها طريقة التّفكير في استخدام المعارف، والمعلومات، والمهارات، بهدف الوصول إلى نتائج لإشباع حاجة الإنسان وزيادة قدراته.\
 أوّل فائدة تستحقّ الذكر هي ما أسهمت بهِ الثورة العلميّة والتكنولوجيّة في مجال مٌكافحة الأمراض، وتطوير المضادّات الحيوية وهو ما يُدعى بالتكنولوجيا الطبيّة الحيويّة، وأيضاُ مساهمة الأجهزة\
 الطبيّة الحديثة في مجالات تخطيط القلب والتصوير الإشعاعي في تشخيص الأمراض، والوصول إلى نتائج مُذهلة في هذا النطاق، أضِف إلى ذلك إدخال التكنولوجيا في إجراء العمليّات الجراحيةّ\
 المعقّدة والدقيقة والجراحة بالمنظار والقسطرة، وكذلك تكنولوجيا صناعة الدواء مما ساعدَ كثيراً في شفاء الحالات المُستعصيّة، وتسهيل العمل على الطاقم البشري من أطبّاء وممرضين وصيادلة.\
 كما نذكُر فائدة التكنولوجيا في التسهيل على الناس في التنقّل من خلال وسائل النقل والمواصلات الحديثة؛ كظهُور القطارات الكهربائية، والطائرات، وأنظمة النقل البحريّ المتطوّرة.\
 وأيضا تساعد التكنولوجيا علي تسهيل مهامّ البحث العلميّ والوصول إلى المعلومة بأقصر وقت وأقلّ تكلفة من السابق؛ حيث توفّر شبكة الإنترنت.", return_tensors="pt")

# Model apply
outputs = model(**inputs)

Some weights of the model checkpoint at asafaya/bert-base-arabic were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
outputs[0]

tensor([[[ 0.0190, -0.8931,  0.8992,  ...,  2.3459,  0.2709, -0.0381],
         [-0.0527,  1.0370,  0.0720,  ...,  0.7532, -0.0681, -0.0105],
         [ 0.5044, -0.0168, -0.2813,  ...,  1.9224,  0.0459, -0.3202],
         ...,
         [-0.5386, -0.6550,  0.2952,  ...,  2.1234,  0.4004,  0.2597],
         [ 0.1163, -0.3460, -0.0468,  ...,  1.6070, -0.1532,  0.2764],
         [ 0.2349, -1.3036,  0.3784,  ...,  1.7620, -0.3244,  0.3365]]],
       grad_fn=<NativeLayerNormBackward0>)

In [29]:
from transformers import AutoTokenizer, AutoModel
import torch
from torch import nn,optim
# Mini:   asafaya/bert-mini-arabic
# Medium: asafaya/bert-medium-arabic
# Base:   asafaya/bert-base-arabic
# Large:  asafaya/bert-large-arabic
model_type = {"Mini": ("asafaya/bert-mini-arabic",256),
                "Medium": ("asafaya/bert-medium-arabic",512),
                "Base": ("asafaya/bert-base-arabic",768),
                "Large": ("asafaya/bert-large-arabic",1024)}
tokenizer = AutoTokenizer.from_pretrained("asafaya/bert-base-arabic")
bert_model = AutoModel.from_pretrained(model_type["Medium"][0])

fc = nn.Linear(model_type["Medium"][1],1)

Downloading: 100%|██████████| 509/509 [00:00<00:00, 255kB/s]
Downloading: 100%|██████████| 162M/162M [00:04<00:00, 34.6MB/s]
Some weights of the model checkpoint at asafaya/bert-medium-arabic were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [41]:
inputs = tokenizer("التكنولوجيا هي كلمة ليست عربية ولكن لها مرادف معرب,\
أقترحه مجمع اللغة العربية بدمشق, واعتمدته الجامعة العربية وبعض الدول العربية, ولكن ليس كلها. و هي كلمة شائعة لها أكثر من مصطلح فالتكنولوجيا \
هي علم تطبيقي يهتم بدراسة الإضافات والتطور في العديد من المجالات مثل الصناعات و الفنون والحرف وكل ما يتعلق بها من مواد ووسائل مستعمل.\
والمفهوم الشّائع لمصطلح التّكنولوجيا هو استعمال الكمبيوتر والأجهزة الحديثة، \
وهذه النّظرة محدودة الرؤية ، فالكمبيوتر نتيجة من نتائج التكنولوجيا،  بينما التكنولوجيا الّتي يقصدها هذا المقرّر هي طريقة للتّفكير، وحلّ المشكلات، وهي أسلوب التّفكير الّذي\
يوصل الفرد إلى النتائج المرجوّة أي إنّها وسيلة وليست نتيجة، وإنّها طريقة التّفكير في استخدام المعارف، والمعلومات، والمهارات، بهدف الوصول إلى نتائج لإشباع حاجة الإنسان وزيادة قدراته.\
 أوّل فائدة تستحقّ الذكر هي ما أسهمت بهِ الثورة العلميّة والتكنولوجيّة في مجال مٌكافحة الأمراض، وتطوير المضادّات الحيوية وهو ما يُدعى بالتكنولوجيا الطبيّة الحيويّة، وأيضاُ مساهمة الأجهزة\
 الطبيّة الحديثة في مجالات تخطيط القلب والتصوير الإشعاعي في تشخيص الأمراض، والوصول إلى نتائج مُذهلة في هذا النطاق، أضِف إلى ذلك إدخال التكنولوجيا في إجراء العمليّات الجراحيةّ\
 المعقّدة والدقيقة والجراحة بالمنظار والقسطرة، وكذلك تكنولوجيا صناعة الدواء مما ساعدَ كثيراً في شفاء الحالات المُستعصيّة، وتسهيل العمل على الطاقم البشري من أطبّاء وممرضين وصيادلة.\
 كما نذكُر فائدة التكنولوجيا في التسهيل على الناس في التنقّل من خلال وسائل النقل والمواصلات الحديثة؛ كظهُور القطارات الكهربائية، والطائرات، وأنظمة النقل البحريّ المتطوّرة.\
 وأيضا تساعد التكنولوجيا علي تسهيل مهامّ البحث العلميّ والوصول إلى المعلومة بأقصر وقت وأقلّ تكلفة من السابق؛ حيث توفّر شبكة الإنترنت.", return_tensors="pt")


TypeError: 'BatchEncoding' object is not callable